In [2]:
import pymunk as pmk
from pymunk.pyglet_util import DrawOptions
import pyglet as pglt
import numpy as np

In [3]:
window = pglt.window.Window(1280,720, "Sim", resizable=False)
options = DrawOptions()
space = pmk.Space()
space.gravity = 0, -100

In [3]:
circle_body = pmk.Body(mass = 1, moment = 1, body_type=pmk.Body.STATIC)
circle_body.position = 450,650
circle_shape = pmk.Circle(circle_body, radius = 30)

circle_body_2 = pmk.Body(mass = 0.1, moment = 1, body_type=pmk.Body.DYNAMIC)
circle_body_2.position = 550,550
circle_shape_2 = pmk.Circle(circle_body_2, radius = 30)

drs = pmk.constraints.DampedRotarySpring(circle_body, circle_body_2, rest_angle=np.pi/2, stiffness = 1000000000000, damping = 1000000)

joint = pmk.constraints.PivotJoint(circle_body, circle_body_2, circle_body.position)

space.add(circle_body, circle_shape, circle_body_2, circle_shape_2, drs, joint)


In [4]:
@window.event
def on_draw():
    window.clear()
    space.debug_draw(options)

def update(dt):
    space.step(dt)

In [ ]:
pglt.clock.schedule_interval(update, 1/160)
pglt.app.run()

Exception ignored on calling ctypes callback function: <function ObjCSubclass.method.<locals>.decorator.<locals>.objc_method at 0x110de8400>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/symbac/lib/python3.12/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 1500, in objc_method
    def objc_method(objc_self, objc_cmd, *args):

KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function ObjCSubclass.method.<locals>.decorator.<locals>.objc_method at 0x110de8400>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/symbac/lib/python3.12/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 1500, in objc_method
    def objc_method(objc_self, objc_cmd, *args):

KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function ObjCSubclass.method.<locals>.decorator.<locals>.objc_method at 0x110de8400>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/symbac/lib/python3.12/site-packages/pyglet/l

In [5]:
help(pmk.Segment)

Help on class Segment in module pymunk.shapes:

class Segment(Shape)
 |  Segment(body: Optional[ForwardRef('Body')], a: tuple[float, float], b: tuple[float, float], radius: float) -> None
 |
 |  A line segment shape between two points.
 |
 |  Meant mainly as a static shape. Can be beveled in order to give them a
 |  thickness.
 |
 |  Method resolution order:
 |      Segment
 |      Shape
 |      pymunk._pickle.PickleMixin
 |      pymunk._typing_attr.TypingAttrMixing
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, body: Optional[ForwardRef('Body')], a: tuple[float, float], b: tuple[float, float], radius: float) -> None
 |      Create a Segment.
 |
 |      It is legal to send in None as body argument to indicate that this
 |      shape is not attached to a body. However, you must attach it to a body
 |      before adding the shape to a space or used for a space shape query.
 |
 |      :param Body body: The body to attach the segment to
 |      :param a: The fir